In [1]:
import pandas as pd

In [2]:
interactions = pd.read_csv("../../Preprocessing/processed_dataframes/interactions.csv")
recipes = pd.read_csv("../../Preprocessing/processed_dataframes/recipes_with_text.csv")
recipes.sort_values(by=["recipe_id"], inplace=True)

Average ratings per recipe

## Train on extended data

In [3]:
import sys
sys.path.append("../../")

n_users = 226570+1
n_items = 231637

In [4]:
from SVD.train_funcs.svd import fit_svd, predict_batch_svd
import numpy as np

def rmse(uir_val, predict_func, **kwargs):
    val_preds = predict_func(uir_val[:, :2], **kwargs)
    val_expected = uir_val[:, 2]

    return np.sqrt(1/len(uir_val) * np.sum((val_preds - val_expected)**2))

train_data = pd.read_csv("alt_extended_train.csv")
validation_data = pd.read_csv("../../Preprocessing/processed_dataframes/val.csv")
uir_train = train_data.values
uir_val = validation_data.values

In [5]:
fitted_params_svd = fit_svd(
    uir_train, uir_val, n_users, n_items, k=4, 
    α1=0.005, α2=0.005, α3=0.005, α4=0.005, λ1=0.01, λ2=0.1,
    n_iters=75,
)

print(rmse(uir_val, predict_batch_svd, params=fitted_params_svd))

  0%|          | 0/75 [00:00<?, ?it/s]

0.8986678334028668


Get items ids that are not rated by the user

In [6]:
my_user_id = interactions["user_id"].max() + 1
my_ratings = pd.read_csv("alt_my_ratings.csv")

In [7]:
unrated_ids = []
for i in range(n_items):
    if i not in my_ratings["recipe_id"].tolist():
        unrated_ids.append(i)

Predict

In [8]:
unrated_ui_mat = np.array([
    [my_user_id]*len(unrated_ids),
    unrated_ids,
]).T

predictions_for_me = predict_batch_svd(unrated_ui_mat, fitted_params_svd)

Top 10 recommended recipes

In [9]:
id_rating_pair_list = [(prediction, recipe_id) for recipe_id, prediction in zip(unrated_ids, predictions_for_me)]
id_rating_pair_list.sort(reverse=True)

In [10]:
for _, recipe_id in id_rating_pair_list[:10]:
    recipe_data = recipes.iloc[recipe_id]
#     print(recipe_data["name"])
    print("Name:", recipe_data["name"])
    print()
    print(", ".join(eval(recipe_data["steps"])))
    print("-"*50)

Name: kittencal s balsamic vinaigrette

in a bowl whisk together garlic , salt , dijon , honey and balsamic vinegar, add in olive oil in a slow steady stream until emulsified, season with more salt if needed and black pepper, chill for a minimum of 2 hours before using
--------------------------------------------------
Name: yummy oatmeal chocolate chip cookies

preheat the oven to 375 degrees f, cream together , using a stand mixer: butter , sugar and brown sugar, add the eggs , one at a time , and the vanilla, blend well, blend 2 1 / 2 cups of the oatmeal in a food processor or blender, in a separate large bowl , mix together: flour , all of the oatmeal , salt , baking powder and baking soda, add the dry mixture , on low - a bit at a time , to the wet ingredients and blend together until fully incorporated, add the chocolate chips and blend in on low, scoop balls onto cookie sheets , spaced a few inches apart, i like to make them about 1 inch balls, bake for about 10 minutes , depend